In [43]:
import yfinance as yf
import pandas as pd
from get_all_tickers import get_tickers as gt #Empresas listadas na NYSE, NASDAQ e AMEX
import sqlite3
import os
import glob
from pathlib import Path

In [30]:
#Local onde está o banco de dados
path_db= 'Companies.db'
#Cria cria banco de dados caso não exista
if os.path.isfile(path) == False:
    !sqlite3 Companies.db < companies_db.schema

In [3]:
#Retorna tickers das empresas listadas na NYSE, NASDAQ e AMEX
symbols = set(gt.get_tickers())
print('Quantidade de empresas listadas: {}'.format(len(symbols)))

Quantidade de empresas listadas: 6445


In [4]:
%%time
conn = sqlite3.connect(path_db)
cursor = conn.cursor()

#Retorna quais empresas já existem no banco de dados
cursor.execute("""
SELECT Ticker FROM Info_gerais;
""")
already_exist = []
for linha in cursor.fetchall():
    already_exist.append(linha[0])
    
new_symbols = symbols.difference(set(already_exist)) #Empresas que ainda não estão no BD

for i, symbol in enumerate(new_symbols):
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        
        #Verifica se os dados desejados existem
        keys = info.keys()
        shortName = info['shortName'] if 'shortName' in keys else None
        market = info['market'] if 'market' in keys else None
        marketCap = info['marketCap'] if 'marketCap' in keys else 'NULL'
        beta = info['beta']if 'beta' in keys else 'NULL'
        country = info['country'] if 'country' in keys else None
        sector = info['sector'] if 'sector' in keys else None
        industry = info['industry'] if 'industry' in keys else None
        
        
        cursor.execute("""
        INSERT INTO Info_gerais (Ticker, [Company name], Market, Marketcap, Beta, Country, Sector, Industry, [Possui dados])
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, FALSE)
        """,(symbol,shortName,market,marketCap,beta,country,sector,industry)
        )
    except:
        cursor.execute("""
        INSERT INTO Info_gerais (Ticker, [Possui dados])
        VALUES (:symbol, FALSE)
        """, {'symbol':symbol} 
        )

        
    print("{}: {}".format(i+1,(len(new_symbols))+1),end="\r")
    
    #Efetua commit a cada 100
    if i >= 100 and i % 100 == 0:
        conn.commit()

#Commit de alterações no banco de dados
conn.commit()

#Termina conexão
conn.close()


CPU times: user 18min 23s, sys: 1min 17s, total: 19min 41s
Wall time: 6h 52min 9s


In [58]:
%%time
conn = sqlite3.connect(path_db)
cursor = conn.cursor()

for file in glob.iglob('./Dados_csv/*.csv'):
    #Coleta nome da planilha a partir do nome do arquivo
    name = Path(file).stem
    
    #csv para DataFrame
    df = pd.read_csv(file)
    
    #Cria string q com n "?," onde é o número de colunas do DataFrame
    q = "".zfill(len(df.columns)).replace("0","?,")[:-1]
    
    for row in df.itertuples():
        cursor.execute('''
                INSERT INTO {table}
                VALUES ({q})
                '''.format(table=name, q=q), row[1:]
                       )
    
    conn.commit()
conn.close()

CPU times: user 5.76 s, sys: 404 ms, total: 6.16 s
Wall time: 6.23 s


In [102]:
%%time
#Checa se upload de dados no BD foi feito corretamente
conn = sqlite3.connect(path_db)
cursor = conn.cursor()

for file in glob.iglob('./Dados_csv/*.csv'):
    #Coleta nome da planilha a partir do nome do arquivo
    name = Path(file).stem
    
    #Cria DataFrame a partir do csv
    df_csv = pd.read_csv(file)
    colunas = df_csv.columns
    #Cria DataFrame a partir do BD
    cursor.execute('''
            SELECT * FROM {table};
            '''.format(table=name)
                   )
    rows = cursor.fetchall()
    df_BD = pd.DataFrame(rows,columns=colunas)
    
    #Compara se ambos Dataframes são iguais
    teste = df_BD.equals(df_csv)
    print("{}: {}".format(name,str(teste)))

conn.close()

Balance_Quartely: True
Growth_Quartely: True
CashFlow_Quartely: True
Income_Quartely: True
CashFlow_Trailing: True
Income_Trailing: True
Metrics_Trailing: True
Metrics_Quartely: True
CPU times: user 8.27 s, sys: 506 ms, total: 8.77 s
Wall time: 8.77 s


In [56]:
# #Deleta todos os valores de uma tabela
# conn = sqlite3.connect(path_db)
# cursor = conn.cursor()

# #Retorna quais empresas já existem no banco de dados
# cursor.execute("""
# DELETE FROM Balance_Quartely;
# """)
# conn.commit()
# conn.close()